## 1-Neighbours manuale

In [1]:
import pandas as pd

In [2]:
iris = pd.read_csv(filepath_or_buffer = r"C:\Users\ianto\Desktop\Corso Python\Lezione 5\iris.csv",
            sep = ",",
            header = 1)

In [3]:
nuovo_iris = [5.9,3.0,5.1,1.8]

In [4]:
iris.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
iris["distanza"] = ((iris["sepal_length"]-nuovo_iris[0])**2 +
(iris["sepal_width"]-nuovo_iris[1])**2 + 
(iris["petal_length"]-nuovo_iris[2])**2 +
(iris["petal_width"]-nuovo_iris[3])**2)**0.5

In [6]:
iris.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species,distanza
0,5.1,3.5,1.4,0.2,setosa,4.140048
1,4.9,3.0,1.4,0.2,setosa,4.153312
2,4.7,3.2,1.3,0.2,setosa,4.298837
3,4.6,3.1,1.5,0.2,setosa,4.149699
4,5.0,3.6,1.4,0.2,setosa,4.173727


In [7]:
x = iris.sort_values(by = "distanza", ascending=True).head(1)

In [8]:
print(x)

     sepal_length  sepal_width  petal_length  petal_width    species  distanza
127           6.1          3.0           4.9          1.8  virginica  0.282843


In [9]:
#oppure
minima_distanza = iris["distanza"].min()
x = iris.query("distanza == @minima_distanza")
print(x)

     sepal_length  sepal_width  petal_length  petal_width    species  distanza
127           6.1          3.0           4.9          1.8  virginica  0.282843


## Usiamo una nostra libreria

In [10]:
import libreria_prevedi_iris as lp

In [11]:
nuovo_iris = [5.9,3.0,5.1,1.8]

In [12]:
lp.prevedi_iris(nuovo_iris)

,sepal_length,sepal_width,petal_length,petal_width,species,distanza
127,6.1,3.0,4.9,1.8,virginica,0.282843


## Usiamo scikit-learn

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [14]:
knc = KNeighborsClassifier(n_neighbors=1)

In [15]:
#per eliminare il warning usare target_training.values.ravel()
knc.fit(X = iris[["sepal_length","sepal_width","petal_length","petal_width"]],
        y = iris[["species"]])

C:\Users\ianto\Desktop\ambiente_python\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=1)

In [16]:
nuovo_iris = pd.DataFrame(data = [nuovo_iris],
                          columns = ["sepal_length","sepal_width","petal_length","petal_width"] )

In [17]:
nuovo_iris

,sepal_length,sepal_width,petal_length,petal_width
0,5.9,3.0,5.1,1.8


In [18]:
knc.predict(X = pd.DataFrame(nuovo_iris))

array(['virginica'], dtype=object)

In [19]:
knc.kneighbors(nuovo_iris)

(array([[0.28284271]]), array([[127]]))

## LLM con Open AI

In [20]:
import pandas as pd
from openai import OpenAI
import os
import getpass

In [21]:
secret_key = getpass.getpass(prompt='Inserisci la chiave: ')

Inserisci la chiave:  ········


In [22]:
os.environ['OPENAI_API_KEY']=secret_key

Inizializziamo le api

In [23]:
client = OpenAI()

Proviamo a fare una chiamata

In [24]:
completition = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "Sei un'assistente AI"},
    {"role": "user", "content": "Come puoi aiutarmi con pandas?"}
  ]
)

In [25]:
print(completition.choices[0].message.content)

Posso aiutarti con diversi aspetti dell'utilizzo di pandas, una libreria potente ed estremamente utile per la manipolazione e l'analisi dei dati in Python. Ecco alcune delle cose con cui posso assisterti:

1. **Installazione**: Ti posso guidare su come installare pandas utilizzando pip o conda.

2. **Importazione dei dati**: Posso spiegarti come leggere diversi tipi di file in un DataFrame pandas, come file CSV, Excel, SQL, JSON, ecc.

3. **Esplorazione dei dati**: Ti posso mostrare come visualizzare le prime righe del DataFrame, ottenere statistiche descrittive e verificare i tipi di dato di ogni colonna.

4. **Manipolazione dei dati**: Posso aiutarti con operazioni di filtraggio, selezione, aggiunta e rimozione di colonne, e riordinamento dei dati.

5. **Pulizia dei dati**: Posso spiegarti come gestire i valori mancanti, rimuovere i duplicati e fare trasformazioni sui dati.

6. **Aggregazione e raggruppamento**: Ti posso guidare su come raggruppare i dati e calcolare aggregati utiliz

## Applicazione pratica con LLM

Acquisiamo il dataframe fatture

In [26]:
fatture = pd.read_csv(filepath_or_buffer = r"C:\Users\ianto\Desktop\Corso python\file\Fatture.csv",
        sep = ";",  #separtore del file
        header = None,
        names = ["NumeroFattura","Tipologia","Importo","Iva","IdCliente","Regione","DataFattura","NumeroFornitore"],
        decimal = ",",
        )

fatture['DataFattura'] =  pd.to_datetime(fatture['DataFattura'],
                                         format="%d/%m/%Y"
                                        )  

fatture['NumeroFornitore'] = fatture['NumeroFornitore'].astype('Int64')

Proviamo a chiedere all'LLM di generare un codice pandas

In [27]:
risposta = client.chat.completions.create(
  model="gpt-4o",
  messages=[
            {"role": "system",
             "content": "Supporti l'utente scrivendo solo codice Python eseguibile con la libreria pandas per rispondere alla sua richiesta" },
            {"role": "user",
             "content": f"""
            Stai lavorando con un dataframe Pandas in Python.
            Il nome del dataframe è df.
            
            Questo è l'elenco delle colonne di df:
            {str(fatture.columns.values)}

            Considera questa richiesta: qual è la fattura più recente?
            
            Segui queste istruzioni:
            1. Converti la richiesta precedente in codice Python eseguibile usando Pandas.
            2. L'ultima riga di codice dovrebbe essere un'espressione Python che può essere chiamata con la funzione `eval()`.
            3. Il codice dovrebbe rappresentare una soluzione alla query.
            4. Restituisci come risposta soltanto il codice Python, non aggiungere altre considerazioni o preamboli
            5. La risposta non deve iniziare con ```python
            6. Non creare dataframe d'esempio
            7. Non fare nessun commento
            8. Rispondi esclusivamente con codice Python
            9. Non usare eval nella risposta
            10. Non creare esempi di dati, usa direttamente nella risposta il nome del dataframe è df 
             """}],
            temperature=0.2)

In [28]:
print(risposta.choices[0].message.content)

import pandas as pd

df['DataFattura'] = pd.to_datetime(df['DataFattura'])
df.loc[df['DataFattura'].idxmax()]


Creiamo una funzione che richiede in input un dataframe e un'interrogazione in linguaggio naturale

In [29]:
def generete_pandas_code(df, query):
    user_content = f"""
            Stai lavorando con un dataframe Pandas in Python.
            Il nome del dataframe è df.
            
            Questo è l'elenco delle colonne di df:
            {str(df.columns.values)}

            Considera questa richiesta: {query}?
            
            Segui queste istruzioni:
            1. Converti la richiesta precedente in codice Python eseguibile usando Pandas.
            2. L'ultima riga di codice dovrebbe essere un'espressione Python che può essere chiamata con la funzione `eval()`.
            3. Il codice dovrebbe rappresentare una soluzione alla query.
            4. Restituisci come risposta soltanto il codice Python, non aggiungere altre considerazioni o preamboli
            5. La risposta non deve iniziare con ```python
            6. Non creare dataframe d'esempio
            7. Non fare nessun commento
            8. Rispondi esclusivamente con codice Python
            9. Non usare eval nella risposta
            10. Non creare esempi di dati, usa direttamente nella risposta il nome del dataframe è df 
            """
    domanda = client.chat.completions.create(
          model="gpt-4o",
          messages=[
                    {"role": "system",
                    "content": "Supporti l'utente scrivendo solo codice Python eseguibile con la libreria pandas per rispondere alla sua richiesta" },
                    {"role": "user",
                     "content":user_content }
                   ])

    risposta = domanda.choices[0].message.content
    return risposta

In [30]:
df_input = fatture

In [31]:
query_input = input("Chiedi la generazione di un codice Pandas. Scrivi x per uscire")

while query_input!="x":
    risposta = generete_pandas_code(df=df_input, query=query_input)
    print(risposta)
    query_input = input("Chiedi la generazione di un codice Pandas. O scrivi x per uscire")

Chiedi la generazione di un codice Pandas. Scrivi x per uscire numero totali di righe


```python
df.shape[0]
```


Chiedi la generazione di un codice Pandas. O scrivi x per uscire x


# LLM in locale

In [32]:
import ollama

Elenco modelli installati in locale

In [33]:
ollama.list()

ListResponse(models=[Model(model='gemma3:1b', modified_at=datetime.datetime(2025, 5, 27, 8, 24, 42, 667464, tzinfo=TzInfo(+02:00)), digest='8648f39daa8fbf5b18c7b4e6a8fb4990c692751d49917417b8842ca5758e7ffc', size=815319791, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='999.89M', quantization_level='Q4_K_M')), Model(model='mistral:latest', modified_at=datetime.datetime(2024, 6, 24, 10, 39, 37, 87989, tzinfo=TzInfo(+02:00)), digest='2ae6f6dd7a3dd734790bbbf58b8909a606e0e7e97e94b7604e0aa7ae4490e6d8', size=4113301090, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='7.2B', quantization_level='Q4_0'))])

Chiamata in locale, può essere molto lenta

In [ ]:
response =  ollama.chat(model="gemma3:1b",
                         messages=[
                                    {"role": "system", "content": "Sei un'assistente AI"},
                                    {"role": "user", "content": "Qual è la capitale dell'italia"}
                                  ]
                       )

In [ ]:
print(response["message"]["content"])

# Python e Database

In [34]:
import getpass
import pandas as pd
from sqlalchemy import create_engine

In [35]:
import json
with open(r"C:\Users\ianto\Desktop\Corso Python\config.json", 'r') as config_file:
    config_data = json.load(config_file)

server_name = config_data['server_name']
database_name = config_data['database_name']
utente = config_data['utente']
password = config_data['password']

#utente = getpass.getpass(prompt='Inserisci nome utente: ')
#password = getpass.getpass(prompt='Inserisci la password: ')

In [36]:
conn_str = f'mssql+pyodbc://{utente}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server&encrypt=yes'

In [37]:
engine = create_engine(conn_str)

In [38]:
df = pd.read_sql( sql = "SELECT TOP 10 * FROM Clienti WHERE Regione = 'Lombardia'", 
             con = engine,
             parse_dates = "DataNascita")
df

,IdCliente,IsActive,Nome,Cognome,DataNascita,Nazione,Regione,Cap,Telefono,Fax,Email,Iniziale2
0,25,True,Cristina,480D8,1991-02-01,Italia,Lombardia,97474,39 320 3143,None,Cristina.480D8@gmail.com,C
1,26,True,Pino,5D47A,1974-03-01,Italia,Lombardia,86929,320 3143,320 3143,Pino.5D47A@gmail.com,P
2,27,True,Vera,4AD27,1973-04-01,Italia,Lombardia,83623,+(39) 320 32312,+(39) 320 32312,Vera.4AD27@gmail.com,V
3,28,True,Milo,55EDA,1973-05-01,Italia,Lombardia,34261,39 320 3143,None,Milo.55EDA@gmail.com,M
4,29,True,Frank,3D18C,1988-06-01,Italia,Lombardia,13596,320 3143,320 3143,Frank.3D18C@gmail.com,F


Scriviamo ora da Python al Database!

In [ ]:
df.to_sql(name = "ClientiPython", 
          con = engine, 
          if_exists="append",
          index=False)

Chiudiamo la connessione

In [40]:
engine.dispose()

Attenzione, per stabilire una connessione crittografata occorre un certificato sul DB. Spesso le stringhe di connessione non hanno questa opzione abilitata di default

In [41]:
import json
with open(r"C:\Users\ianto\Desktop\Corso Python\config_2.json", 'r') as config_file:
    config_data = json.load(config_file)

server_name = config_data['server_name']
database_name = config_data['database_name']
utente = config_data['utente']
password = config_data['password']

#utente = getpass.getpass(prompt='Inserisci nome utente: ')
#password = getpass.getpass(prompt='Inserisci la password: ')

In [42]:
conn_str = f'mssql+pyodbc://{utente}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server&encrypt=yes'

In [43]:
engine = create_engine(conn_str)

La prossima istruzione restituisce l'errore


Restituisce errore
OperationalError: (pyodbc.OperationalError) ('08001', "[08001] [Microsoft][ODBC Driver 17 for SQL Server]Provider SSL: Catena di certificati emessa da una Autorità di certificazione non disponibile nell'elenco locale.\r\n (-2146893019) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Il client non è in grado di effettuare la connessione. (-2146893019)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
df = pd.read_sql( sql = "SELECT TOP 10 * FROM Clienti WHERE Regione = 'Lombardia'", 
             con = engine,
             parse_dates = "DataNascita")
df

In [44]:
engine.dispose()